# LangChain Expression Language (LCEL)

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
#!pip install pydantic==1.10.8

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [4]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

/Users/pierrebittner/miniforge3/envs/finaxys-training-genai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
chain = prompt | model | output_parser

In [6]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [7]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [8]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

/Users/pierrebittner/miniforge3/envs/finaxys-training-genai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/pierrebittner/miniforge3/envs/finaxys-training-genai/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [9]:
retriever.get_relevant_documents("where did harrison work?")

/Users/pierrebittner/miniforge3/envs/finaxys-training-genai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [10]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [11]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [12]:
from langchain.schema.runnable import RunnableMap

In [13]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [14]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [15]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [16]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind

and OpenAI Functions

In [17]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [19]:
runnable = prompt | model

In [20]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 64, 'total_tokens': 80}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-f45e74c0-4447-4bfe-b0a0-574c3c716de5-0')

In [21]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [22]:
model = model.bind(functions=functions)

In [23]:
runnable = prompt | model

In [24]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 99, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-f1cd2e38-d2ba-4ae8-a12a-cb887c223b77-0')

## Fallbacks

In [25]:
from langchain.llms import OpenAI
import json

In [32]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="davinci-002"
)
simple_chain = simple_model | json.loads

In [33]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [34]:
simple_model.invoke(challenge)

' of the poem. The poems should be randomly selected from a list of 100 poems. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a random order. The poems should be displayed in a rand

Note: The next line is expected to fail.

In [35]:
simple_chain.invoke(challenge)

JSONDecodeError: Expecting value: line 1 column 2 (char 1)

In [36]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [37]:
chain.invoke(challenge)

{'poem1': {'title': 'The Night Sky',
  'author': 'Emily Dickinson',
  'firstLine': 'The night is starry and the stars are blue.'},
 'poem2': {'title': 'Autumn Leaves',
  'author': 'Robert Frost',
  'firstLine': "My sorrow, when she's here with me, thinks these dark days of autumn rain are beautiful as days can be."},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul.'}}

In [38]:
final_chain = simple_chain.with_fallbacks([chain])

In [39]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Night Sky',
  'author': 'Emily Dickinson',
  'firstLine': 'The night is starry and the stars are blue and shiver in the distance.'},
 'poem2': {'title': 'Autumn Leaves',
  'author': 'Robert Frost',
  'firstLine': "My sorrow, when she's here with me, thinks these dark days of autumn rain are beautiful as days can be."},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul, and sings the tune without the words, and never stops at all.'}}

## Interface

In [40]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [41]:
chain.invoke({"topic": "bears"})

'Why did the bear bring a flashlight to the party? \n\nBecause he wanted to be the light of the bear-becue!'

In [42]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears like fast food? Because they can't catch it!",
 'Why are frogs so good at basketball? Because they always make jump shots!']

In [43]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 don
't
 bears
 like
 fast
 food
?
 Because
 they
 can
't
 catch
 it
!



In [44]:
response = await chain.ainvoke({"topic": "bears"})
response

'Why did the bear bring a flashlight to the party? \n\nBecause he wanted to be the "light" of the party!'